### Тестовое задание в Jetbrains Research
https://research.jetbrains.org/ru-ru/groups/biolabs/

Скачайте таблицу с данными по methylation level цитозинов https://artyomovlab.wustl.edu/publications/supp_materials/aging/rrbs/filtered_cytosines_freq.tsv.gz. Данные приведены для сборки генома hg19. Первый столбец соответствует позиции в геноме, остальные уровню метилирования в соответствующем доноре.

Требуется выполнить пункты ниже для размера бина 100bp, 1kbp, 10kbp, 100kpb, 1mb и в итоге получить 5 BigWig треков.

- Разбейте геном на непересекающиеся бины заданной длинны. Например, chr1:0-1000, chr1:1000-2000, chr1:2000-3000,.... для размера бина 1 kbp.
- Для каждого бина в каждом доноре вычислите средний уровень метилирования. За средний уровень метилирования в бине можно принять среднее арифметическое уровня метилирования по всем цитозинам, для которых есть данные.
- Таким образом, каждому бину можно сопоставить вектор среднего уровня метилирования по всем донорам. Посчитайте пирсоновскую корреляцию между данными в бине и в соседнем бине справа.
- Сгенерируйте BigWig трек, где в каждом бине будет отображаться значение посчитанной корреляции, и все бины будут сдвинуты на половину длина бина вправо (на границу между соседними бинами). Например, для бинов chr1:0-1000, chr1:1000-2000, chr1:2000-3000,.... нужно получить трек с значениями коэф корреляции в регионах: chr1:500-1500, chr1:1500-2500, chr1:2500-3500, .., где региону chr1:1500-2500 соответствует коэф. корреляции векторов chr1:1000-2000, chr1:2000-3000.

#### P.S поясню про биологическую часть задания и датасет, чтобы было немного понятнее.
Одна из характеристик генома - это уровень его метилирования, то есть в каких местах и в каких количествах метиловый остаток присоединен к основаниям ДНК. Метилирование влияет на активность генов, а общий уровень метилирования коррелирует с процессами старения. Данный датасет - пример того, какие используются в большом исследовании, направленном на изучение изменения уровня метилирования генома с возрастом человека. Конкретно эта тестовая задача направлена на выявление корреляций уровня метилирования в соседних регионах генома, что может помочь построить модель, прогнозирующую уровень метилирования в регионах, данных о которых в исследовании может не быть(естественно, не конкретно моя реализация, а задача в целом).

Датасет состоит из колонки, в которой указан номер хромосомы и позиция в ней, а в остальных - процент метилирования, полученный при введении разных метиловых доноров (молекул, отдающих метиловый остаток) в клетку  - то есть какой процент введенных метилов был присоеден к этой конкретной позиции в ДНК. 

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

bins = [10**2, 10**3, 10**4, 10**5, 10**6]

In [3]:
df = pd.read_csv("./data/filtered_cytosines_freq.tsv", sep='\t')
df.head()

,chrBase,OD10,OD11,OD12,OD13,OD14,OD15,OD16,OD17,OD18,...,YD20,YD21,YD2,YD3,YD4,YD5,YD6,YD7,YD8,YD9
0,chr1.10636,100.000,NaN,NaN,0.0,100.000,NaN,100.000,NaN,88.889,...,NaN,85.714,97.368,NaN,100.0,0.0,NaN,92.982,97.222,90.000
1,chr1.10617,88.000,NaN,NaN,0.0,100.000,NaN,100.000,NaN,88.889,...,NaN,85.714,92.308,NaN,100.0,50.0,0.0,86.792,100.000,60.000
2,chr1.10638,100.000,NaN,NaN,0.0,95.652,NaN,100.000,NaN,88.889,...,NaN,85.714,97.436,NaN,100.0,0.0,NaN,96.491,100.000,81.818
3,chr1.10631,100.000,NaN,NaN,NaN,100.000,NaN,100.000,NaN,88.889,...,NaN,85.714,97.436,NaN,100.0,50.0,0.0,96.154,97.297,81.818
4,chr1.10609,95.455,NaN,NaN,0.0,100.000,NaN,63.636,NaN,88.889,...,NaN,85.714,94.737,NaN,100.0,100.0,NaN,96.296,97.222,80.000


In [4]:
df.columns

Index(['chrBase', 'OD10', 'OD11', 'OD12', 'OD13', 'OD14', 'OD15', 'OD16',
       'OD17', 'OD18', 'OD19', 'OD1', 'OD20', 'OD2', 'OD3', 'OD4', 'OD5',
       'OD6', 'OD7', 'OD8', 'OD9', 'YD10', 'YD11', 'YD12', 'YD14', 'YD15',
       'YD16', 'YD17', 'YD18', 'YD19', 'YD1', 'YD20', 'YD21', 'YD2', 'YD3',
       'YD4', 'YD5', 'YD6', 'YD7', 'YD8', 'YD9'],
      dtype='object')

#### Проверим, есть ли в колонках уровня типы данных, отличные от float или nan, которые могут испортить вычисления. Их нет

In [5]:
df.dtypes

chrBase     object
OD10       float64
OD11       float64
OD12       float64
OD13       float64
OD14       float64
OD15       float64
OD16       float64
OD17       float64
OD18       float64
OD19       float64
OD1        float64
OD20       float64
OD2        float64
OD3        float64
OD4        float64
OD5        float64
OD6        float64
OD7        float64
OD8        float64
OD9        float64
YD10       float64
YD11       float64
YD12       float64
YD14       float64
YD15       float64
YD16       float64
YD17       float64
YD18       float64
YD19       float64
YD1        float64
YD20       float64
YD21       float64
YD2        float64
YD3        float64
YD4        float64
YD5        float64
YD6        float64
YD7        float64
YD8        float64
YD9        float64
dtype: object

#### Заметим, что макс уровень метилирования на всех донорах - 100, значит данные приведены в процентах

In [6]:
df.iloc[:, 1:41].max().max()

100.0

#### Проверим, нет ли дупликатов среди позиций. Дупликатов нет, значит их дропать не нужно

In [7]:
df[df.duplicated(['chrBase'])]

,chrBase,OD10,OD11,OD12,OD13,OD14,OD15,OD16,OD17,OD18,...,YD20,YD21,YD2,YD3,YD4,YD5,YD6,YD7,YD8,YD9


#### Заметим, что chrBase не осортирована по порядку внутри каждой хромосомы, отсортируем по возрастанию по номеру хромосомы и позиции. Для этого создадим колонки chr и pos, chr в виде численных значений, причем хромосомы X, Y и M  будут иметь номера 23, 24, и 25 cоответсвенно для удобства сортировки. 

In [8]:
def split_chrom_position(v):
    l = v.split('.')
    l[0] = l[0][3:]
    try:
        l[0] = int(l[0])
    except:
        if l[0] == 'X':
            l[0] = 23
        elif l[0] == 'Y':
            l[0] = 24
        else:
            l[0] = 25
    l[1] = int(l[1])
    return l

df[['chr', 'pos']] = df.chrBase.apply(
    lambda v: split_chrom_position(v)).to_list()

df_sorted = df.sort_values(['chr', 'pos'], ignore_index=True)
df_sorted

,chrBase,OD10,OD11,OD12,OD13,OD14,OD15,OD16,OD17,OD18,...,YD2,YD3,YD4,YD5,YD6,YD7,YD8,YD9,chr,pos
0,chr1.10497,71.956,93.640,94.410,92.163,86.408,92.143,91.304,69.029,76.023,...,85.714,73.761,84.277,80.408,94.667,92.623,83.992,89.620,1,10497
1,chr1.10525,93.704,90.813,95.679,89.375,93.528,95.714,93.103,89.737,92.941,...,91.081,91.329,92.332,85.537,92.857,89.617,92.073,94.684,1,10525
2,chr1.10542,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,...,100.000,100.000,100.000,100.000,99.647,99.708,99.781,100.000,1,10542
3,chr1.10589,100.000,NaN,NaN,0.000,100.000,NaN,100.000,NaN,88.889,...,92.308,NaN,100.000,50.000,0.000,91.379,91.892,81.818,1,10589
4,chr1.10609,95.455,NaN,NaN,0.000,100.000,NaN,63.636,NaN,88.889,...,94.737,NaN,100.000,100.000,NaN,96.296,97.222,80.000,1,10609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808443,chrM.16452,0.000,0.000,4.124,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.610,4.688,5.263,0.000,0.000,2.899,1.695,25,16452
2808444,chrM.16456,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.407,0.000,0.000,25,16456
2808445,chrM.16457,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,2.424,4.478,5.172,0.000,0.000,2.817,0.000,25,16457
2808446,chrM.16497,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.810,0.000,0.000,25,16497


### Пример вычислений для размера бина 1kbp

#### Каждую хромосому разобъем на бины длиной 1kbp. Добавим к каждой позиции бин, которому она принадлежит

In [9]:
chr_group = df_sorted.groupby(['chr'])

bin_group = chr_group.apply(lambda grp: pd.cut(grp.pos, bins=pd.interval_range(
    start=0, freq=bins[1], end=grp.pos.max()+bins[1], closed='left'))).reset_index(drop=True)

df_sorted['bin_interval'] = bin_group
df_sorted

,chrBase,OD10,OD11,OD12,OD13,OD14,OD15,OD16,OD17,OD18,...,YD3,YD4,YD5,YD6,YD7,YD8,YD9,chr,pos,bin_interval
0,chr1.10497,71.956,93.640,94.410,92.163,86.408,92.143,91.304,69.029,76.023,...,73.761,84.277,80.408,94.667,92.623,83.992,89.620,1,10497,"[10000, 11000)"
1,chr1.10525,93.704,90.813,95.679,89.375,93.528,95.714,93.103,89.737,92.941,...,91.329,92.332,85.537,92.857,89.617,92.073,94.684,1,10525,"[10000, 11000)"
2,chr1.10542,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,...,100.000,100.000,100.000,99.647,99.708,99.781,100.000,1,10542,"[10000, 11000)"
3,chr1.10589,100.000,NaN,NaN,0.000,100.000,NaN,100.000,NaN,88.889,...,NaN,100.000,50.000,0.000,91.379,91.892,81.818,1,10589,"[10000, 11000)"
4,chr1.10609,95.455,NaN,NaN,0.000,100.000,NaN,63.636,NaN,88.889,...,NaN,100.000,100.000,NaN,96.296,97.222,80.000,1,10609,"[10000, 11000)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808443,chrM.16452,0.000,0.000,4.124,0.000,0.000,0.000,0.000,0.000,0.000,...,0.610,4.688,5.263,0.000,0.000,2.899,1.695,25,16452,"[16000, 17000)"
2808444,chrM.16456,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.407,0.000,0.000,25,16456,"[16000, 17000)"
2808445,chrM.16457,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,2.424,4.478,5.172,0.000,0.000,2.817,0.000,25,16457,"[16000, 17000)"
2808446,chrM.16497,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.810,0.000,0.000,25,16497,"[16000, 17000)"


#### Сгруппируем данные по хромосоме и бину. Вычислим средний уровень метилирования для каждого бина в каждом доноре. Уберем колонку pos, т.к она больше не нужна

In [10]:
df_binned_mean = df_sorted.groupby(['chr', 'bin_interval'], as_index=False).mean().drop('pos', 1)
df_binned_mean

,chr,bin_interval,OD10,OD11,OD12,OD13,OD14,OD15,OD16,OD17,...,YD20,YD21,YD2,YD3,YD4,YD5,YD6,YD7,YD8,YD9
0,1,"[10000, 11000)",94.616545,94.817667,96.696333,28.15380,97.780727,95.952333,95.276636,86.255333,...,94.293333,86.696364,94.375636,88.363333,97.873545,51.449545,35.896375,93.399273,96.063727,83.944909
1,1,"[133000, 134000)",96.179500,100.000000,100.000000,97.12775,89.527750,91.014000,95.671250,84.440750,...,97.222250,95.656500,90.576250,95.056000,96.124750,93.440000,94.599000,96.030000,91.884500,92.155000
2,1,"[134000, 135000)",97.561000,100.000000,100.000000,100.00000,88.235000,100.000000,97.674000,84.091000,...,98.485000,93.878000,94.444000,87.500000,100.000000,100.000000,88.000000,100.000000,100.000000,94.915000
3,1,"[135000, 136000)",86.585000,97.753000,95.154500,81.81800,96.875000,93.089000,96.511500,100.000000,...,99.231000,95.902500,95.827000,62.500000,87.750000,95.833500,91.576000,90.672500,98.077000,90.241000
4,1,"[136000, 137000)",73.119400,60.000000,76.470600,72.62780,79.166600,74.228600,77.108000,79.633400,...,68.571600,76.959800,76.697400,70.426200,76.568000,77.753600,75.296800,76.277400,75.887600,72.517200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284180,25,"[11000, 12000)",0.238143,1.298714,0.000000,0.00000,0.000000,0.000000,2.083333,0.000000,...,7.142857,0.000000,0.207000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.432857
284181,25,"[12000, 13000)",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.041750,0.000000,0.000000,0.000000
284182,25,"[13000, 14000)",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.625000,17.307750,...,0.000000,0.000000,0.595250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
284183,25,"[15000, 16000)",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,2.564000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526000,0.000000,0.000000


#### Посчитаем пирсоновскую корреляцию между данными в бине и в соседнем бине справа. Для этого транспонируем датафрейм, а его копию шифтнем на -1. corrwith посчитает корреляцию между всеми колонками на одинаковых позициях, что даст нам нужную корреляцию между бинами

In [11]:
df_binned_mean_T = df_binned_mean.T
df_binned_mean_T_shifted = df_binned_mean_T.shift(-1, axis=1)
bin_corrs = df_binned_mean_T.iloc[2:].astype('float64').corrwith(df_binned_mean_T_shifted.iloc[2:].astype('float64'), axis=0)

#### Удалим корреляции, посчитанные для последнего бина одной хромосомы, и первого бина следующей. Для этого найдем индексы последних бинов в каждой хромосоме, кроме последней, и поставим NaN строку c данным индексом из таблицы корреляций

In [12]:
corr_ids_to_del = [df_binned_mean[df_binned_mean['chr'] == i].index.max() for i in df_binned_mean['chr'].unique()[:-1]]
bin_corrs[bin_corrs.index.isin(corr_ids_to_del)] = np.nan
bin_corrs

0         0.105618
1         0.403661
2         0.211492
3         0.079279
4        -0.175575
            ...   
284180   -0.085663
284181   -0.059692
284182   -0.043207
284183   -0.098942
284184         NaN
Length: 284185, dtype: float64

#### Почему столько nan в корреляциях: 83 ряда полностью состоят из нулей, корреляции соседних бинов с ними всегда будут nan. То есть минимум 166 корреляций будет nan. ещё +1 корреляция -nan, потому что 2 ряда подряд идут со всеми нулями.  +25 корреляций -  nan, потому что это отброшенные корреляции между последним бином одной хромосомы и первым бином другой, и последняя (из-за сдвига). Итого это 191 корреляции. Почему ещё около 200 корреляций - nan, пока точно выяснить не удалось. Так как pandas игнорирует nan при подсчете mean или corrwith, несколько nanов в ряду на итоговую корреляцию не влияют. Возможно, это связано с конкретными наборами значений в этих конректных бинах, то есть какие-то комбинации нулей и nanoв в 2ух рядах влияют на то, что корреляция между ними будет nan. Так как природа значения nan по метилированию в исходном датасете мне не известна, а доля таких корреляций - меньше 1%, считаю, что это в рамках задания можно игнорировать.
P.S Для размера бина 10^6 nan корреляций (за искл отброшенных и M-хромосомы) вообще нет, а для 10^2 - процент nan корреляций - 1%. То есть для данной задачи есть более и менее оптимальные разбиения бинов.

In [13]:
bin_corrs.isna().sum()

410

In [14]:
subset = df_binned_mean.loc[:, 'OD10':'YD9']
all_zeros = subset[(subset.T == 0).all()]
print('number of rows that have all zeros: ', len(all_zeros))

if len(all_zeros) != 0:
    paired = [(all_zeros.index[i], all_zeros.index[i+1]) for i in range(len(all_zeros)-1)]
    res = list(filter(lambda e: e==1, [v[1]-v[0] for v in paired]))
    print('number of all-zeros rows that are adjacent: ', len(res))

all_nans = subset[(subset.T.isna()).all()]
print('number of all-nans rows: ', len(all_nans))

nan_corr_percentage = ((bin_corrs.isna().sum()-len(corr_ids_to_del)-1)/len(bin_corrs))*100
print('nan-correlation %: ', nan_corr_percentage)

number of rows that have all zeros:  83
number of all-zeros rows that are adjacent:  1
number of all-nans rows:  0
nan-correlation %:  0.1354751306367331


#### Запишем корреляции для соотв бинов

In [15]:
df_binned_mean['p_corr'] = bin_corrs
df_binned_mean.tail(10)

,chr,bin_interval,OD10,OD11,OD12,OD13,OD14,OD15,OD16,OD17,...,YD21,YD2,YD3,YD4,YD5,YD6,YD7,YD8,YD9,p_corr
284175,25,"[0, 1000)",0.022727,0.030364,0.010545,0.000000,0.051091,0.066818,0.061000,0.204091,...,0.599545,0.000000,0.447909,0.026091,0.130636,0.035636,0.157091,0.105818,0.033727,0.302937
284176,25,"[3000, 4000)",0.031625,0.087375,0.056000,0.200250,0.012875,0.000000,0.181125,0.174000,...,0.393250,0.012250,1.590125,0.049500,0.123250,0.069250,0.151625,0.144500,0.042125,NaN
284177,25,"[4000, 5000)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
284178,25,"[6000, 7000)",0.059111,0.202333,0.031222,0.101111,0.279111,0.087125,0.023444,0.564571,...,0.028667,0.045111,0.905111,0.047778,0.210556,1.262111,0.204778,0.245778,0.317333,0.258609
284179,25,"[8000, 9000)",0.057667,0.347167,0.126333,0.559667,0.687000,0.102833,0.000000,0.935167,...,0.011167,0.078000,0.756167,0.344333,0.264167,0.031833,0.306167,0.295833,1.208833,-0.143739
284180,25,"[11000, 12000)",0.238143,1.298714,0.000000,0.000000,0.000000,0.000000,2.083333,0.000000,...,0.000000,0.207000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.432857,-0.085663
284181,25,"[12000, 13000)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.041750,0.000000,0.000000,0.000000,-0.059692
284182,25,"[13000, 14000)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.625000,17.307750,...,0.000000,0.595250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.043207
284183,25,"[15000, 16000)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.564000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526000,0.000000,0.000000,-0.098942
284184,25,"[16000, 17000)",0.000000,0.000000,2.717714,0.000000,0.376000,0.000000,0.000000,0.000000,...,0.639429,0.000000,0.682571,2.525286,3.007857,0.000000,0.276571,2.015429,0.242143,NaN


#### Cдвинем бины на половину бина вправо

In [16]:
df_binned_mean['bin_interval'] = df_binned_mean['bin_interval'].map(lambda i: i+(bins[1]/2))
df_binned_mean

,chr,bin_interval,OD10,OD11,OD12,OD13,OD14,OD15,OD16,OD17,...,YD21,YD2,YD3,YD4,YD5,YD6,YD7,YD8,YD9,p_corr
0,1,"[10500.0, 11500.0)",94.616545,94.817667,96.696333,28.15380,97.780727,95.952333,95.276636,86.255333,...,86.696364,94.375636,88.363333,97.873545,51.449545,35.896375,93.399273,96.063727,83.944909,0.105618
1,1,"[133500.0, 134500.0)",96.179500,100.000000,100.000000,97.12775,89.527750,91.014000,95.671250,84.440750,...,95.656500,90.576250,95.056000,96.124750,93.440000,94.599000,96.030000,91.884500,92.155000,0.403661
2,1,"[134500.0, 135500.0)",97.561000,100.000000,100.000000,100.00000,88.235000,100.000000,97.674000,84.091000,...,93.878000,94.444000,87.500000,100.000000,100.000000,88.000000,100.000000,100.000000,94.915000,0.211492
3,1,"[135500.0, 136500.0)",86.585000,97.753000,95.154500,81.81800,96.875000,93.089000,96.511500,100.000000,...,95.902500,95.827000,62.500000,87.750000,95.833500,91.576000,90.672500,98.077000,90.241000,0.079279
4,1,"[136500.0, 137500.0)",73.119400,60.000000,76.470600,72.62780,79.166600,74.228600,77.108000,79.633400,...,76.959800,76.697400,70.426200,76.568000,77.753600,75.296800,76.277400,75.887600,72.517200,-0.175575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284180,25,"[11500.0, 12500.0)",0.238143,1.298714,0.000000,0.00000,0.000000,0.000000,2.083333,0.000000,...,0.000000,0.207000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.432857,-0.085663
284181,25,"[12500.0, 13500.0)",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.041750,0.000000,0.000000,0.000000,-0.059692
284182,25,"[13500.0, 14500.0)",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.625000,17.307750,...,0.000000,0.595250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.043207
284183,25,"[15500.0, 16500.0)",0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,2.564000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526000,0.000000,0.000000,-0.098942


#### Отформатируем таблицу для записи в bed файл. Также уберем бины, в которых корреляция nan, так как это невалидное значение для этих файлов. Можно загружать эти файлы напрямую в геномный браузер, а можно сгененирить из них файлы формата bigWig с помощью юниксовой утилиты.

In [17]:
df_to_bed = df_binned_mean.loc[:, ['chr', 'bin_interval', 'p_corr']]

def rename_chrom(c):
    if c == 23:
        c = 'X'
    elif c == 24:
        c = 'Y'
    elif c == 25:
        c = 'M'
    else: 
        pass
    return 'chr{}'.format(c)

df_to_bed['chr'] = df_to_bed.chr.apply(lambda c: rename_chrom(c))

df_to_bed[['bin_left', 'bin_right']] = df_to_bed['bin_interval'].apply(
    lambda r: [int(r.left), int(r.right)]).to_list()

df_to_bed = df_to_bed.drop('bin_interval', 1)
df_to_bed = df_to_bed.loc[:, ['chr', 'bin_left', 'bin_right', 'p_corr']]

df_to_bed.tail(10)
df_to_bed = df_to_bed.dropna()

df_to_bed.to_csv('{}_bin_corr.bed'.format(bins[1]), index=False, header = False, sep='\t')